In [11]:
!rm -r model1

In [3]:
import os
import random
import time

import numpy as np
import pandas as pd

from matplotlib import pyplot
from PIL import Image,ImageTk

from tkinter import Tk,Frame,Label,IntVar,Radiobutton,Canvas,Button

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [4]:
from sklearn.cluster import KMeans

embeds_file = "embedings.npy"

embeds = np.load(embeds_file)

num_clusters = 50

In [14]:
cluster = KMeans(num_clusters, random_state=13)
cluster.fit(embeds)

df = pd.read_csv("embedings.csv")
img_paths = df["img_path"]

labels = cluster.labels_

In [15]:
img_paths.head()

0    ./16k_images/B06XBYTSNM.jpeg
1    ./16k_images/B074W35Z7F.jpeg
2    ./16k_images/B01LWJWB64.jpeg
3    ./16k_images/B01MDK2HIR.jpeg
4    ./16k_images/B072DZS2YB.jpeg
Name: img_path, dtype: object

In [16]:
print(embeds.shape)

(15990, 1280)


In [17]:
os.makedirs("./model1/clusters")
for i in range(num_clusters):
    os.makedirs("./model1/clusters/cluster "+str(i),exist_ok=True)
    for j in range(embeds.shape[0]):
        if labels[j] == i:
            img = Image.open(img_paths[j])
            img.save("./model1/clusters/cluster "+str(i)+"/"+str(j)+".jpeg")

In [5]:
prob_dis = []
a = [0 for i in range(num_clusters)]
prob_dis.extend(a)

prob_dis = np.array(prob_dis)
prob_cdf = np.cumsum(prob_dis)

prob_cdf.shape

(40,)

In [6]:
win = Tk()
win.geometry("1920x1080")

v1 = IntVar(win, 1)
v2 = IntVar(win, 1)
v3 = IntVar(win, 1)
v4 = IntVar(win, 1)

clusters = [0 for i in range(4)]

started = False

out_chance = 0.1

net_penality = 1
net_reward = 1
penality = 1
reward = 1

l1 = 0.7
l2 = 0.85
l3 = 0.97
l4 = 1

labels = list()

times = list()

def show_suggestions():
    start = time.time()

    images = list()

    for lab in labels:
        lab.destroy()

    ind = np.argpartition(prob_dis,-4)[-4:]
    #print(clusters,ind)
    if started:
        #print(prob_dis)
        
        vs = [v1.get(), v2.get(), v3.get(), v4.get()]
        
        for i in range(4):
            if vs[i] == 1:
                prob_dis[clusters[i]] += reward
            elif vs[i] == 2:
                prob_dis[clusters[i]] -= penality

    radio_values = {"Positive" : 1, "Negative" : 2, "Neutral" : 3}

    i = 0
    for _ in range(4):
        num = np.random.uniform()
        #print(num)
        if num <= out_chance:
            clusters[i] = np.random.randint(0,num_clusters - 1)
        else:
            num = np.random.uniform()

            if num < l1:
                clusters[i] = ind[0]
            elif num < l2:
                clusters[i] = ind[1]
            elif num < l3:
                clusters[i] = ind[2]
            else:
                clusters[i] = ind[3]

        cluster = clusters[i]

        images = os.listdir("./model1/clusters/cluster " + str(cluster) + "/")
        img = Image.open("./model1/clusters/cluster " + str(cluster) + "/" + random.choice(images))

        img_gui = ImageTk.PhotoImage(img)

        frame = Label(win, image=img_gui)
        frame.image = img_gui

        if i == 0:
            frame.grid(row = 0, column = 0, rowspan = 3)

            count = 0
            for (text,value) in radio_values.items():
                Radiobutton(win, text=text, variable=v1, value=value).grid(row = count, column = 1)
                count += 1
        elif i == 1:
            frame.grid(row = 0, column = 2, rowspan = 3)

            count = 0
            for (text,value) in radio_values.items():
                Radiobutton(win, text=text, variable=v2, value=value).grid(row = count, column = 3)
                count += 1
        elif i == 2:
            frame.grid(row = 3, column = 0, rowspan = 3)

            count = 3
            for (text,value) in radio_values.items():
                Radiobutton(win, text=text, variable=v3, value=value).grid(row = count, column = 1)
                count += 1
        else:
            frame.grid(row = 3, column = 2, rowspan = 3)

            count = 3
            for (text,value) in radio_values.items():
                Radiobutton(win, text=text, variable=v4, value=value).grid(row = count, column = 3)
                count += 1

        labels.append(frame)

        i += 1

        end = time.time()

        times.append(end - start)

show_suggestions()

started = True

btn = Button(win, command=show_suggestions, text="Submit", anchor="center")
btn.grid(row=0,column=4, rowspan=6)

win.mainloop()

times = np.array(times)

print("Average time per frame:", times.mean())
print("Median time for frame:", np.median(times))

Average time per frame: 0.02404072880744934
Median time for frame: 0.020844101905822754
